In [1]:
from pathlib import Path
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon

In [3]:
# Paths
directorio_actual = Path.cwd()
dir_modis = directorio_actual/"modis"

In [4]:
# Datos vectoriales de paises
datos_boundaries_n0 = directorio_actual/"boundaries/ne_50m_admin_0_countries/ne_50m_admin_0_countries.shp"

# Cargar la capa vectorial con los datos del país
gdf_paises = gpd.read_file(datos_boundaries_n0)
gdf_paises.head(3)

,featurecla,scalerank,LABELRANK,SOVEREIGNT,SOV_A3,ADM0_DIF,LEVEL,TYPE,TLC,ADMIN,...,FCLASS_TR,FCLASS_ID,FCLASS_PL,FCLASS_GR,FCLASS_IT,FCLASS_NL,FCLASS_SE,FCLASS_BD,FCLASS_UA,geometry
0,Admin-0 country,1,3,Zimbabwe,ZWE,0,2,Sovereign country,1,Zimbabwe,...,None,None,None,None,None,None,None,None,None,"POLYGON ((31.28789 -22.40205, 31.19727 -22.344..."
1,Admin-0 country,1,3,Zambia,ZMB,0,2,Sovereign country,1,Zambia,...,None,None,None,None,None,None,None,None,None,"POLYGON ((30.39609 -15.64307, 30.25068 -15.643..."
2,Admin-0 country,1,3,Yemen,YEM,0,2,Sovereign country,1,Yemen,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((53.08564 16.64839, 52.58145 16..."


In [5]:
# Sample for add contry to fire data points
files_br = ["DL_FIRE_M-C61_469416/fire_archive_M-C61_469416.csv","DL_FIRE_M-C61_469910/fire_archive_M-C61_469910.csv"]
files_out =["fire_archive_M-C61_469416","fire_archive_M-C61_469910"]

In [8]:
for index,file in enumerate(files_br):
    print("processing file: ",file)
    dir_modis_world_x = dir_modis/file
    dfx = pd.read_csv(dir_modis_world_x)
    print(dfx.head())
    gdf_fuegox = gpd.GeoDataFrame(dfx, geometry=gpd.points_from_xy(dfx.longitude, dfx.latitude), crs="EPSG:4326")

    # Hacer una unión espacial entre los puntos de fuego y los países para obtener el nombre del país para cada punto
    gdf_fuego_con_paisesx = gpd.sjoin(gdf_fuegox, gdf_paises[['SOVEREIGNT', 'geometry']], how="left", op="within")

    # Renombrar la columna resultante
    gdf_fuego_con_paisesx.rename(columns={'SOVEREIGNT': 'country_name'}, inplace=True)
    gdf_fuego_con_paisesx.to_parquet(f"{files_out[index]}_mod.parquet")
    print("created parquet")
    gdf_fuego_con_paisesx.to_csv(f"{files_out[index]}_mod.csv", index=False)
    print("created csv")
    print(gdf_fuego_con_paisesx.head())

processing file:  DL_FIRE_M-C61_469416/fire_archive_M-C61_469416.csv
   latitude  longitude  brightness  scan  track    acq_date  acq_time  \
0  -22.2797   -41.7369       301.4   1.0    1.0  2016-05-13       131   
1  -22.2783   -41.7272       313.8   1.0    1.0  2016-05-13       131   
2  -22.2873   -41.7257       300.2   1.0    1.0  2016-05-13       131   
3  -20.2325   -40.2385       304.9   1.1    1.1  2016-05-13       132   
4  -18.8706   -39.8467       302.1   1.2    1.1  2016-05-13       132   

  satellite instrument  confidence  version  bright_t31   frp daynight  type  
0     Terra      MODIS          41     6.03       289.2   4.4        N     2  
1     Terra      MODIS          88     6.03       289.4  12.5        N     2  
2     Terra      MODIS          22     6.03       289.5   3.6        N     2  
3     Terra      MODIS          47     6.03       290.5   6.6        N     2  
4     Terra      MODIS          47     6.03       288.5   8.0        N     0  


/home/vant/anaconda3/envs/geopython/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


created parquet
created csv
   latitude  longitude  brightness  scan  track    acq_date  acq_time  \
0  -22.2797   -41.7369       301.4   1.0    1.0  2016-05-13       131   
1  -22.2783   -41.7272       313.8   1.0    1.0  2016-05-13       131   
2  -22.2873   -41.7257       300.2   1.0    1.0  2016-05-13       131   
3  -20.2325   -40.2385       304.9   1.1    1.1  2016-05-13       132   
4  -18.8706   -39.8467       302.1   1.2    1.1  2016-05-13       132   

  satellite instrument  confidence  version  bright_t31   frp daynight  type  \
0     Terra      MODIS          41     6.03       289.2   4.4        N     2   
1     Terra      MODIS          88     6.03       289.4  12.5        N     2   
2     Terra      MODIS          22     6.03       289.5   3.6        N     2   
3     Terra      MODIS          47     6.03       290.5   6.6        N     2   
4     Terra      MODIS          47     6.03       288.5   8.0        N     0   

                      geometry  index_right country_

/home/vant/anaconda3/envs/geopython/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


created parquet
created csv
   latitude  longitude  brightness  scan  track    acq_date  acq_time  \
0   -4.8607   -65.2966       302.9   1.4    1.2  2011-01-01       307   
1  -23.1440   -47.1176       302.7   1.0    1.0  2011-01-01       433   
2   -3.3957   -41.2370       317.9   4.4    1.9  2011-01-01      1352   
3   -3.3935   -41.2315       317.0   4.4    1.9  2011-01-01      1352   
4  -21.4961   -48.1797       317.7   1.5    1.2  2011-01-01      1643   

  satellite instrument  confidence  version  bright_t31    frp daynight  type  \
0     Terra      MODIS          52     6.03       291.5    5.7        N     0   
1      Aqua      MODIS          51     6.03       288.6    5.8        N     0   
2     Terra      MODIS          72     6.03       288.6  110.5        D     0   
3     Terra      MODIS          55     6.03       287.3   96.8        D     0   
4      Aqua      MODIS          49     6.03       297.4   13.5        D     0   

                      geometry  index_right co